In [1]:
import numpy as np
import pandas as pd
from scipy.special import softmax

In [7]:
df=pd.read_csv('chatbot_dataset.csv')
df.head(2)

,User ID,User Utterance,Bot Response,Timestamp,Context/Session ID,Entities,User Feedback,Conversation Outcome,User Profile,Channel/Platform,Language,User Emotion/Sentiment,Location,User Segment
0,uKqYhMMQ5S,Charge bar between follow student.,Important law into large example range. Player...,2023-10-31 18:02:06,4dfe56dc-efe2-49c6-be9f-ce5b84ca4de4,event,negative,incomplete,Annette Henderson,social media,German,confused,Sydney,returning customers
1,YOonrpgxp9,Bad every reflect huge contain.,Policy argue agree character go recent. When r...,2023-10-31 18:02:06,74c97616-9df6-460d-991d-c6f6f6ae5354,location,neutral,specific outcome,Nicholas Haney,mobile app,Chinese,frustrated,London,returning customers


In [11]:
x=df['User Utterance']
y=df['Bot Response']

In [15]:
y[0]

'Important law into large example range. Player seem force with partner sometimes happen southern.'

In [16]:
x[0]

'Charge bar between follow student.'

In [18]:
vocab=["My name is Bablu","Bablu goes to college"]

In [19]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(vocab)

In [20]:
sequences=tokenizer.texts_to_sequences(vocab)

In [21]:
sequences

[[2, 3, 4, 1], [1, 5, 6, 7]]

In [22]:
tokenizer.word_index

{'bablu': 1, 'my': 2, 'name': 3, 'is': 4, 'goes': 5, 'to': 6, 'college': 7}

In [17]:
# 4 dimensions for each word
import math
def positionalEncodings(sequence):
    
    PE=[]
    d_model=4
    for pos in range(sequence):
        EncodingsOfEach=[]
        for i in range(d_model):
            if i%2==0:
                temp=pos/(math.pow(10000,(2*i)/d_model))
                EncodingsOfEach.append(math.sin(temp))
            else:
                temp=pos/(math.pow(10000,(2*(i-1))/d_model))
                EncodingsOfEach.append(math.cos(temp))
        PE.append(EncodingsOfEach)
    return PE

In [18]:
tokens = ["The", "cat", "sat"]
PE=(positionalEncodings(len(tokens)))


In [19]:
PE

[[0.0, 1.0, 0.0, 1.0],
 [0.8414709848078965, 0.5403023058681398, 9.999999983333334e-05, 0.999999995],
 [0.9092974268256817,
  -0.4161468365471424,
  0.0001999999986666667,
  0.9999999800000001]]

In [4]:
a=[1,0,1,0]
b=np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
result=np.matmul(a,b)
result

array([1, 0, 1, 0])

In [5]:
a=np.array([1,0,1,0])
b=np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
result=np.matmul(a,b)
result
print(a.shape)
print(b.shape)
print(result.shape)


(4,)
(4, 4)
(4,)


In [6]:
#input will be the embedding(+PE) 
# eg:[[1,0,1,0],[0,1,0,1],[1,1,1,1]]
def selfAttention(embeddings,W_q, W_k,W_v):
    numberOfWords=len(embeddings)
    Q_K_V_ofAllWords=[]
    for i in range(numberOfWords):
        Q_of_given_word=np.matmul(embeddings[i],W_q)
        K_of_given_word=np.matmul(embeddings[i],W_k)
        V_of_given_word=np.matmul(embeddings[i],W_v)
        temp=[]
        temp.append(Q_of_given_word)
        temp.append(K_of_given_word)
        temp.append(V_of_given_word)
        Q_K_V_ofAllWords.append(temp)
    d_k=math.sqrt(len(Q_K_V_ofAllWords[0][1]))
    scoreOfAllWords=[]
    for i in range(numberOfWords):
        query=Q_K_V_ofAllWords[i][0]
        scoreOfEachWord=[]
        for j in range(numberOfWords):
            key=Q_K_V_ofAllWords[j][1]
            scoreOfEachWord.append(np.matmul(query,key)/d_k)
        scoreOfAllWords.append(scoreOfEachWord)
    attentionWeightsOfAllWords=[]
    for i in range(numberOfWords):
        attentionWeightsOfAllWords.append(softmax(scoreOfAllWords[i]))
    
    finalAttentionOutput=[]
    for i in range(numberOfWords):
        d_v = len(Q_K_V_ofAllWords[0][2])
        temp = np.zeros(d_v)
        for j in range(numberOfWords):
            temp += attentionWeightsOfAllWords[i][j] * Q_K_V_ofAllWords[j][2]
        finalAttentionOutput.append(temp)
    return finalAttentionOutput

    
    


In [ ]:
temp_matrix=np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
embeddings=np.array([[1,0,1,0],[0,1,0,1],[1,1,1,1]])
self_atten_out=selfAttention(embeddings,temp_matrix,temp_matrix,temp_matrix)

In [63]:
self_atten_out

[array([0.8446376, 0.5776812, 0.8446376, 0.5776812]),
 array([0.5776812, 0.8446376, 0.5776812, 0.8446376]),
 array([0.78805844, 0.78805844, 0.78805844, 0.78805844])]

In [85]:
def Add_and_normalize(output_of_previous,input_of_previous):
    #added=output_of_previous+input_of_previous
    output=[]
    eps = 1e-6
    for i in range(len(output_of_previous)):
        added=output_of_previous[i]+input_of_previous[i]
        mean = np.mean(added)
        std_dev = np.std(added)
        normalized_array=(added-mean)/(std_dev+eps)
        output.append(normalized_array)
    return output

    

In [86]:
add_and_normalize_1_out=(Add_and_normalize(self_atten_out,embeddings))

In [87]:
add_and_normalize_1_out

[array([ 0.99999842, -0.99999842,  0.99999842, -0.99999842]),
 array([-0.99999842,  0.99999842, -0.99999842,  0.99999842]),
 array([0., 0., 0., 0.])]

In [88]:
def FeedForwardNN(x,W1,b1,W2,b2):
    output=[]
    for i in range(len(x)):
        hidden=np.maximum(0,np.matmul(x[i],W1)+b1)
        out=np.matmul(hidden,W2)+b2
        output.append(out)
    return output





In [89]:
dimOfInput=add_and_normalize_1_out[0].shape[0]
W1=np.random.random((dimOfInput,256))
b1=np.random.random((256,))
W2=np.random.random((256,dimOfInput))
b2=np.random.random((dimOfInput,))
ffn_out=FeedForwardNN(add_and_normalize_1_out,W1,b1,W2,b2)
ffn_out

[array([74.44746168, 76.26869145, 74.69544957, 73.8961129 ]),
 array([83.22672236, 81.79718994, 88.32028904, 80.69721868]),
 array([67.6595549 , 69.42100484, 71.3437252 , 65.49451367])]

In [90]:
encoder_out=Add_and_normalize(ffn_out,add_and_normalize_1_out)
encoder_out

[array([ 0.55157821,  0.39267561,  0.77200986, -1.71626368]),
 array([-0.57464065, -0.31926194,  1.70558656, -0.81168396]),
 array([-0.37959203,  0.43566937,  1.32557244, -1.38164978])]

In [105]:
def Encoder(positional_encodings_plus_embeddings,W_q,W_k,W_v,W1,b1,W2,b2):
    self_attn_output=selfAttention(positional_encodings_plus_embeddings,W_q,W_k,W_v)
    add_and_normalize_1_output=Add_and_normalize(self_attn_output,positional_encodings_plus_embeddings)
    ffn_output=FeedForwardNN(add_and_normalize_1_output,W1,b1,W2,b2)
    encoder_output=Add_and_normalize(ffn_output,add_and_normalize_1_output)
    return encoder_output
    

In [106]:
embeddings=np.array([[1,0,1,0],[0,1,0,1],[1,1,1,1]])
positional_encodings=positionalEncodings(len(embeddings))
embeddings_plus_PE=embeddings+positional_encodings

In [107]:
d_model = embeddings_plus_PE.shape[1]

W_q = np.eye(d_model)
W_k = np.eye(d_model)
W_v = np.eye(d_model)

W1 = np.random.randn(d_model, 256)
b1 = np.zeros(256)
W2 = np.random.randn(256, d_model)
b2 = np.zeros(d_model)


In [108]:
print(Encoder(embeddings_plus_PE,W_q,W_k,W_v,W1,b1,W2,b2))

[array([ 0.4625303 ,  0.8084736 ,  0.44262381, -1.71362771]), array([-0.35390072,  0.59077209,  1.20400299, -1.44087436]), array([ 0.59019507,  1.05767249, -0.05736201, -1.59050555])]
